# Train YOLO

In [ ]:
!pip install ultralytics

!wget -q https://github.com/direito-a-sombra/bus-view/releases/latest/download/imgs.tar.gz
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/data/training/train_boxes.json
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/data/training/train_files.json
!tar -xzf imgs.tar.gz

In [ ]:
import json

from os import makedirs
from shutil import copy2
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

In [ ]:
# Get info about all training image files

IMG_DIR = "./imgs"
TRAINING_FILES_JSON = "./data/training/train_files.json"

with open(TRAINING_FILES_JSON, "r") as ifp:
  training_files = json.load(ifp)

id2path = {}
for label, d2fs in training_files.items():
  for d ,fs in d2fs.items():
    for f in fs:
      id2path[f"{d}/{f}"] = { "dir": d, "name": f }

In [ ]:
# Prep Yolo dataset

DATASET_DIR = "./datasets/bus-view"
makedirs(f"{DATASET_DIR}/images/train", exist_ok=True)
makedirs(f"{DATASET_DIR}/images/val", exist_ok=True)
makedirs(f"{DATASET_DIR}/labels/train", exist_ok=True)
makedirs(f"{DATASET_DIR}/labels/val", exist_ok=True)

LABEL2ID = {
  "bus_stop": 0,
  "bus_sign": 1,
}
ID2LABEL = { id:label for label,id in LABEL2ID.items() }

with open("./data/training/train_boxes.json", "r") as ifp:
  box_info = json.load(ifp)

tids, vids = train_test_split(list(box_info.keys()), test_size=0.2, random_state=1010)

tids = set(tids)
vids = set(vids)

for img_id, objs in box_info.items():
  split = "train" if img_id in tids else "val"
  img_dir = id2path[img_id]["dir"]
  img_name = id2path[img_id]["name"]
  yolo_id = f"{img_dir}_{img_name}"

  src_file = f"{IMG_DIR}/{img_dir}/{img_name}"
  dst_file = f"{DATASET_DIR}/images/{split}/{yolo_id}"
  copy2(src_file, dst_file)

  with open(f"{DATASET_DIR}/labels/{split}/{yolo_id.replace('.jpg', '.txt')}", "w") as ofp:
    for label, (x0,y0,x1,y1) in objs.items():
      cx, cy = int((x0 + x1) // 2) / 640, int((y0 + y1) // 2) / 640
      w, h = abs(int(x1 - x0)) / 640, abs(int(y1 - y0)) / 640
      ofp.write(f"{LABEL2ID[label]} {cx} {cy} {w} {h}\n")

with open(f"./bus-view.yaml", "w") as ofp:
  ofp.write(f"path: bus-view\n")
  ofp.write(f"train: images/train\n")
  ofp.write(f"val: images/val\n\n")
  ofp.write(f"names:\n")
  for id,label in ID2LABEL.items():
    ofp.write(f"  {id}: {label}\n")

In [ ]:
model = YOLO("yolo11x.pt")

In [ ]:
results = model.train(data="bus-view.yaml", epochs=64, imgsz=(640, 640), device="cuda")